In [6]:
# this line makes figures interactive in Jupyter notebooks
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import cantera as ct

from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity

# for convenience:
def to_si(quant):
    '''Converts a Pint Quantity to magnitude at base SI units.
    '''
    return quant.to_base_units().magnitude

In [3]:
# these lines are only for helping improve the display
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['figure.dpi']= 150
plt.rcParams['savefig.dpi'] = 150

In [47]:
#full_species = {S.name: S for S in ct.Species.listFromFile('nasa_gas.cti')}

#species = [full_species[S] for S in (
#    'N2H4', 'N2', 'H2', 'H', 'N', 'NH'
#    )]
#gas = ct.Solution(thermo='IdealGas', species=species)

#gas = ct.Solution('n2h4.yaml')
gas = ct.Solution('nasa_n2h4.cti')

temperature = Q_(5000, 'K')
pressure = Q_(50, 'psi')

gas.TPX = to_si(temperature), to_si(pressure), 'N2H4:1.0'
#gas.equilibrate('TP')
#gas()
#print(Q_(gas.P, 'Pa').to('bar'))

# based on example https://cantera.org/examples/python/thermo/sound_speed.py.html

# set the gas to equilibrium at its current T and P
rtol=1.0e-6
maxiter=5000
gas.equilibrate('TP', rtol=rtol, max_iter=maxiter)

# save properties
X0 = gas.X
s0 = gas.s
p0 = gas.P
h0 = gas.h
T0 = gas.T
r0 = gas.density

print(f'gamma=cp/cv: {gas.cp/gas.cv}')

# perturb the pressure
p1 = p0*1.0001

# set the gas to a state with the same entropy and composition but
# the perturbed pressure
gas.SP = s0, p1

# frozen sound speed
afrozen = np.sqrt((p1 - p0)/(gas.density - r0))
gamma_frozen = (np.log(p1) - np.log(p0)) / (np.log(gas.density) - np.log(r0))

# now equilibrate the gas holding S and P constant
gas.equilibrate('SP', rtol=rtol, max_iter=maxiter)

# equilibrium sound speed
aequil = np.sqrt((p1 - p0)/(gas.density - r0))
gamma_equil = (np.log(p1) - np.log(p0)) / (np.log(gas.density) - np.log(r0))

# compute the frozen sound speed using the ideal gas expression as a check
gamma = gas.cp/gas.cv
afrozen2 = np.sqrt(gamma * ct.gas_constant * gas.T /
                   gas.mean_molecular_weight)

gas.TPX = T0, p0, X0
T1 = T0 * 1.0001
gas.TP = T1, p0
gas.equilibrate('SP', rtol=rtol, max_iter=maxiter)
cp_equil = (gas.h - h0) / (gas.T - T0)
print(f'C_p,equil = {cp_equil}')

print(gamma_frozen, gamma_equil, gamma)
print(afrozen, aequil, afrozen2)

gamma=cp/cv: 1.4919307570740907
C_p,equil = 11090.553125494333
1.4919186582279218 1.2550230291153577 1.4919329172682332
3048.5785541452924 2796.0780576325237 3048.598968608774


In [25]:
gas.cp_mass / gas.cv_mass

1.4919307570740903

In [42]:
gas.cp_mass / (gas.cp_mass - ct.gas_constant/gas.mean_molecular_weight)

1.4919309728029861

In [43]:
gas.cp_mole / (gas.cp_mole - ct.gas_constant)

1.491930972802986